# Part 1: Data Collection

#### https://mersakarya.medium.com/selenium-tutorial-scraping-glassdoor-com-in-10-minutes-3d0915c6d905
#### Selenium Tutorial: Scraping Glassdoor.com in 10 Minutes by Omer Sakarya
#### https://github.com/arapfaik/scraping-glassdoor-selenium/blob/master/glassdoor%20scraping.ipynb

In [1]:
# pip install selenium

In [2]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd

In [3]:
jobs = []
num_jobs = 10
closeSocial = False
keyword = 'data scientist'
path = "C:/Users/User/Documents/intro_data_science/chromedriver"
url = "https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword="+keyword+"&sc.keyword="+keyword+"&locT=&locId=&jobType="
slp_time=8

In [ ]:
def closeSocialLogin(driver):
    try:
        driver.find_element_by_css_selector('[alt="Close"]').click() 
    except NoSuchElementException:
        pass

def closeSignUpPrompt(driver):
    try:
        driver.find_element_by_class_name("selected").click()
    except:
        pass

In [ ]:
def initWebDdriver(webdriver):
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(executable_path=path, options=options)
    driver.set_window_size(1120, 800)
    driver.get(url)
    print("webdriver initiated...")
    return driver

In [ ]:
job_listing = "li.react-job-listing.job-search-key-nhtksm"

glassdoor_job_description = [
    ['company_name', '//*[@id="JDCol"]/div/article/div/div[1]/div/div/div[1]/div/div[1]/div[1]'],
    ['location', '//*[@id="JDCol"]/div/article/div/div[1]/div/div/div[1]/div/div[1]/div[3]' ],
    ['job_title', '//*[@id="JDCol"]/div/article/div/div[1]/div/div/div[1]/div/div[1]/div[2]'],
    ['job_description', '//*[@id="JobDescriptionContainer"]'],
    ['salary_estimate', '//*[@id="JDCol"]/div/article/div/div[2]/div[1]/div[2]/div/div[1]/div[2]/div[1]'],
    ['rating','//*[@id="employerStats"]/div[1]/div[1]'],
    ['size', '//*[@id="EmpBasicInfo"]/div[1]/div/div[1]/span[2]' ],
    ['founded', '//*[@id="EmpBasicInfo"]/div[1]/div/div[2]'],
    ['type_of_ownership', '//*[@id="EmpBasicInfo"]/div[1]/div/div[2]'],
    ['industry', '//*[@id="EmpBasicInfo"]/div[1]/div/div[4]/span[2]'],
    ['sector','//*[@id="EmpBasicInfo"]/div[1]/div/div[5]'],
    ['revenue','//*[@id="EmpBasicInfo"]/div[1]/div/div[3]'],
]

In [ ]:
def pullJobDescription(driver, glassdoor_xpaths):
    print("starting pull...")
    job_info={}
    for i in range(len(glassdoor_xpaths)):
        try:
            job_info[glassdoor_xpaths[i][0]] = driver.find_element_by_xpath(glassdoor_xpaths[i][1]).text
        except:
            job_info[glassdoor_xpaths[i][0]] = -1
    print(job_info)
    return job_info

In [ ]:
driver = initWebDdriver(webdriver)
    
while len(jobs) < num_jobs:  
    # wait for page to be loaded
    time.sleep(slp_time)
    closeSignUpPrompt(driver)        
    job_buttons = driver.find_elements_by_css_selector(job_listing)
        
    for job_button in job_buttons:
        print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
        if len(jobs) >= num_jobs:
            break
        #job_button.click()  
        driver.execute_script("arguments[0].click();", job_button)
        time.sleep(1)
        closeSocialLogin(driver)
        time.sleep(1)
        job_info = pullJobDescription(driver, glassdoor_job_description)
        jobs.append(job_info)
        
    try:
        driver.find_element_by_xpath('//*[@id="MainCol"]/div[2]/div/div[1]/button[7]/span/svg').click() #next page
    except NoSuchElementException:
        print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
        break


In [ ]:
df = pd.DataFrame(jobs).reset_index()
df.to_csv('Uncleaned_DS_jobs.csv', index=False)

In [ ]:
df